In [37]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [38]:
# Step 1: Create an imbalanced class binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [39]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [40]:
#Experiment 1: Train Logistic Regression Classifier
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [41]:
# Experiment 2: Train Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.63      0.76        30

    accuracy                           0.96       300
   macro avg       0.96      0.81      0.87       300
weighted avg       0.96      0.96      0.96       300



In [42]:
#Experiment 3: Train XGBoost
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [43]:
#Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [44]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [45]:
#Track Experiments Using MLFlow this code is same as the above code for creating model obj, fit and predict and classification.
# defines the model and stores it with the data. No training or fit or prediction occurs yet.
# for all other things like creating model obj, fit, prediction and classification we are doing it through a for loop below.

models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE", #model_name
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),  #model
        (X_train_res, y_train_res), # (train_set[0], train_set[1])
        (X_test, y_test) # (test_set[0], test_set[1])
    )
]

In [46]:
# for training, testing datasets, fit, predict and classification reports im using a for loop, and running through each model.
# then appending to an empty 'reports = []' list.
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [47]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [48]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

'''
    # In this code,
    (
        "Random Forest",  # this is element[0] ie, model_name
        RandomForestClassifier(n_estimators=30, max_depth=3),  # this is element[1] ie, model
        (X_train, y_train),
        (X_test, y_test)
    ),
'''

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name): # we can see in mlflow ui each model is named its own model_name.
        mlflow.log_param("model", model_name) # setting parameters to show in mlflow ui
        mlflow.log_metric('accuracy', report['accuracy']) # setting metrices to show in mlflow ui or we can pass a single dictionary of all the metrices.
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/12/06 23:28:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/06 23:29:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/4/runs/66c797bb24b145fb83bc3a69a12332a5
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/06 23:29:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/4/runs/5b0ae381d64c4094b7e425b68e6efab0
🧪 View experiment at: http://localhost:5000/#/experiments/4


2025/12/06 23:29:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/4/runs/7987009a05114d7081bf90705979cb1b
🧪 View experiment at: http://localhost:5000/#/experiments/4
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/4/runs/4e426eb442f848f3b6f354c7e98c1082
🧪 View experiment at: http://localhost:5000/#/experiments/4
